In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistics model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    #Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

def get_keypoints(results):
    try:
        pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    except AttributeError:
        pose = np.zeros(132)

    try:
        face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()
    except AttributeError:
        face = np.zeros(468*3)

    try:
        lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    except AttributeError:
        lh = np.zeros(63)

    try:
        rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    except AttributeError:
        rh = np.zeros(63)

    return np.concatenate([pose, face, lh, rh])


Creating data folders

In [3]:
DATA_PATH = os.path.join('ASL_ALPHABET')

alphabets = np.array(['A', 'B', 'C'])

no_sequences, sequence_length = 30,30

In [12]:
for alphabet in alphabets:
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, alphabet, str(sequence)))
        except:
            pass

In [34]:
cap = cv2.VideoCapture(0)
alpha_visted = set()
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for alphabet in alphabets:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)

                draw_styled_landmarks(image, results)

                if frame_num == 0:

                    if alphabet not in alpha_visted:
                        cv2.putText(image, 'STARTING \'{}\' COLLECTION'.format(alphabet), (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                        alpha_visted.add(alphabet)
                    else:     
                       cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for Alphabet \'{}\' Video Number: {}'.format(alphabet, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000) 

                else:
                    cv2.putText(image,'Collecting frames for Alphabet \'{}\' Video Number: {}'.format(alphabet, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                np.save(os.path.join(DATA_PATH, alphabet, str(sequence), str(frame_num)), get_keypoints(results))
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [44]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [38]:
alpha_labels = {alpha:label for label, alpha in enumerate(alphabets)}

In [39]:
alpha_labels

{'A': 0, 'B': 1, 'C': 2}

In [41]:
sequences, labels = [],[]
for alpha in alphabets:
    for seq in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, alpha, str(seq), '{}.npy'.format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(alpha_labels[alpha])

In [56]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow

In [64]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [65]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(alphabets.shape[0], activation='softmax'))

In [67]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [68]:
model.fit(X_train, y_train, epochs=5000, callbacks=[tb_callback])

0.9374
Epoch 2387/5000
3/3 [==============================] - 0s 103ms/step - loss: 0.3424 - categorical_accuracy: 0.9491
Epoch 2388/5000
3/3 [==============================] - 0s 111ms/step - loss: 0.3412 - categorical_accuracy: 0.8963
Epoch 2389/5000
3/3 [==============================] - 0s 96ms/step - loss: 0.3379 - categorical_accuracy: 0.9413
Epoch 2390/5000
3/3 [==============================] - 0s 102ms/step - loss: 0.3084 - categorical_accuracy: 0.9354
Epoch 2391/5000
3/3 [==============================] - 0s 113ms/step - loss: 0.3285 - categorical_accuracy: 0.9198
Epoch 2392/5000
3/3 [==============================] - 0s 106ms/step - loss: 0.3604 - categorical_accuracy: 0.8552
Epoch 2393/5000
3/3 [==============================] - 0s 135ms/step - loss: 0.3429 - categorical_accuracy: 0.9100
Epoch 2394/5000
3/3 [==============================] - 0s 119ms/step - loss: 0.3567 - categorical_accuracy: 0.8297
Epoch 2395/5000
3/3 [==============================] - 0s 159ms/step - los

KeyboardInterrupt: 

In [69]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_13 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

In [70]:
res = model.predict(X_test)

In [73]:
alphabets[np.argmax(res[1])],alphabets[np.argmax(y_test[1])]

('B', 'B')

In [74]:
model.save('abc.h5')

In [75]:
from sklearn.metrics import multilabel_confusion_matrix

In [77]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 1]],

       [[2, 0],
        [0, 3]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [78]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, alphabets[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [86]:
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        # print(result)
        draw_styled_landmarks(image, results)

        keypoints = get_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(alphabets[np.argmax(res)])
            predictions.append(np.argmax(res))

            # print(res[np.argmax(res)])

            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if alphabets[np.argmax(res)] != sentence[-1]:
                            sentence.append(alphabets[np.argmax(res)])
                    else:
                        sentence.append(alphabets[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]

            image = prob_viz(res, alphabets, image, colors)
            

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
C
C
A
A
A
A
A
A
A
A
A
A
A
A
A
C
C
C
B
B
B
B
C
B
C
C
C
C
C
C
C
C
C
C
C
A
A
A
A
A
A
A
A
A
A
C
B
B
C
C
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
C
C
C
C
B
C
C
B
B
B
B
C
B
C
B
C
C
C
B
A
A
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
C
C
C
B
C
C
C
C
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
C
A
C
A
A
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
A
C
C
A
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
B
B
B
B
B
B
B
B
B
B
B
C
A
A
A
A
A
A
A
C
B
B
B
B
B
B
B
C
C
A
A


In [84]:
cap.release()
cv2.destroyAllWindows()